In [23]:
# Initial imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
# Load .env environment variables
load_dotenv()
import datetime as dt
import pytz

_timeNow = dt.datetime.now(pytz.timezone('US/Eastern'))
_1YearsAgo = _timeNow - dt.timedelta(days=360)


API_KEY = os.getenv("ALPACA_API_KEY")
API_SECRET = os.getenv("ALPACA_SECRET_KEY")
ALPACA_API_BASE_URL = "https://paper-api.alpaca.markets"

# Create a connection to the API 
api = tradeapi.REST(API_KEY, API_SECRET, ALPACA_API_BASE_URL, api_version="v2")

# Set the ticket symbol and the number of shares to buy
ticker = "SQ"
number_of_shares = 1

# Make API call
prices = api.get_bars(ticker, 'day',
                         start=_1YearsAgo.isoformat(),
                         end=None,
                         limit=None).df
prices.columns

Index(['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'], dtype='object')

In [24]:
# Slice to just the `close` column
signals_df = prices.loc[:,["close"]]


In [25]:
# Set the short window and long windows
short_window = 50
long_window = 100

In [26]:
# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()

# Prepopulate the `Signal` for trading
signals_df['Signal'] = 0.0
signals_df = signals_df.dropna()
signals_df

,close,SMA50,SMA100,Signal
timestamp,,,,
2021-10-28 08:30:00+00:00,255.10,259.306674,258.971992,0.0
2021-10-28 09:30:00+00:00,255.60,259.033074,259.000492,0.0
2021-10-28 11:00:00+00:00,256.34,258.799874,259.036392,0.0
2021-10-28 11:30:00+00:00,256.15,258.619674,259.071892,0.0
2021-10-28 12:00:00+00:00,256.22,258.457074,259.106992,0.0
...,...,...,...,...
2022-10-17 18:30:00+00:00,55.95,53.979308,54.757082,0.0
2022-10-17 19:00:00+00:00,55.81,53.985308,54.787682,0.0
2022-10-17 19:30:00+00:00,55.95,53.979708,54.808682,0.0


In [27]:
# Generate the trading signal 0 or 1,
# where 1 is when short-window (SMA50) is greater than the long (SMA 100)
# and 0 otherwise
signals_df['Signal'] = np.where(
    signals_df['SMA50']> signals_df['SMA100'], 1.0, 0.0
)

In [28]:
signals_df

,close,SMA50,SMA100,Signal
timestamp,,,,
2021-10-28 08:30:00+00:00,255.10,259.306674,258.971992,1.0
2021-10-28 09:30:00+00:00,255.60,259.033074,259.000492,1.0
2021-10-28 11:00:00+00:00,256.34,258.799874,259.036392,0.0
2021-10-28 11:30:00+00:00,256.15,258.619674,259.071892,0.0
2021-10-28 12:00:00+00:00,256.22,258.457074,259.106992,0.0
...,...,...,...,...
2022-10-17 18:30:00+00:00,55.95,53.979308,54.757082,0.0
2022-10-17 19:00:00+00:00,55.81,53.985308,54.787682,0.0
2022-10-17 19:30:00+00:00,55.95,53.979708,54.808682,0.0


In [29]:
##### Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Review the DataFrame
# Review the DataFrame
signals_df.loc["2015-2-11":"2019-10-30",:]

,close,SMA50,SMA100,Signal,Entry/Exit
timestamp,,,,,


In [31]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Create the overlay plot
entry_exit_plot = entry * exit

# Show the plot with a title
entry_exit_plot.opts(
    title="SQ - SMA50, SMA100, Entry and Exit Points"
)

:Overlay
   .Scatter.Close.I  :Scatter   [timestamp]   (close)
   .Scatter.Close.II :Scatter   [timestamp]   (close)

In [32]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

In [33]:
# Buy a 500 share position when the dual moving average crossover Signal equals 1
# Otherwise, `Position` should be zero (sell)
signals_df['Position'] = share_size * signals_df['Signal']

In [34]:
# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [35]:

# Determine the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

In [36]:
# Multiply the close price by the number of shares held, or the Position
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Position']

# Subtract the amount of either the cost or proceeds of the trade from the initial capital invested
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum() 

In [37]:

# Calculate the total portfolio value by adding the portfolio cash to the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Conclusion:  each point at a time, you can see what you have is 
# 1) if you are holding stock. say 500 share, what's holding value of 500 share at that moment + (initial capital + net of cost and proceed)
# in each row, you have  holding = position * close + initial investment - cost of previous buys and + proceed of previous sales



In [38]:
signals_df.loc["2015-12-1":"2016-1-30",:]

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total
timestamp,,,,,,,,,,


In [39]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)
# Visualize the value of the total portfolio
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Apple Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [timestamp]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [timestamp]   (Portfolio Total)